In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pointbiserialr
from math import sqrt

In [2]:
def getMerit(subset, label):
    k = len(subset)

    # average feature-class correlation
    rcf_all = []
    for feature in subset:
        coeff = pointbiserialr( df[label], df[feature] )
        rcf_all.append( abs( coeff.correlation ) )
    rcf = np.mean( rcf_all )

    # average feature-feature correlation
    corr = df[subset].corr()
    corr.values[np.tril_indices_from(corr.values)] = np.nan
    corr = abs(corr)
    rff = corr.unstack().mean()

    return (k * rcf) / sqrt(k + k * (k-1) * rff)

In [3]:
class PriorityQueue:
    def  __init__(self):
        self.queue = []

    def isEmpty(self):
        return len(self.queue) == 0
    
    def push(self, item, priority):
        """
        item already in priority queue with smaller priority:
        -> update its priority
        item already in priority queue with higher priority:
        -> do nothing
        if item not in priority queue:
        -> push it
        """
        for index, (i, p) in enumerate(self.queue):
            if (set(i) == set(item)):
                if (p >= priority):
                    break
                del self.queue[index]
                self.queue.append( (item, priority) )
                break
        else:
            self.queue.append( (item, priority) )
        
    def pop(self):
        # return item with highest priority and remove it from queue
        max_idx = 0
        for index, (i, p) in enumerate(self.queue):
            if (self.queue[max_idx][1] < p):
                max_idx = index
        (item, priority) = self.queue[max_idx]
        del self.queue[max_idx]
        return (item, priority)

In [4]:
def cfs(features, labels, df):
    best_value = -1
    best_feature = ''
    for feature in features:
        coeff = pointbiserialr( df[label], df[feature] )
        abs_coeff = abs( coeff.correlation )
        if abs_coeff > best_value:
            best_value = abs_coeff
            best_feature = feature

    # initialize queue
    queue = PriorityQueue()
    # push first tuple (subset, merit)
    queue.push([best_feature], best_value)
    # list for visited nodes
    visited = []
    # counter for backtracks
    n_backtrack = 0
    # limit of backtracks
    max_backtrack = 5
    
    # repeat until queue is empty
    # or the maximum number of backtracks is reached
    while not queue.isEmpty():
        # get element of queue with highest merit
        subset, priority = queue.pop()

        # check whether the priority of this subset
        # is higher than the current best subset
        if (priority < best_value):
            n_backtrack += 1
        else:
            best_value = priority
            best_subset = subset

        # goal condition
        if (n_backtrack == max_backtrack):
            break

        # iterate through all features and look of one can
        # increase the merit
        for feature in features:
            temp_subset = subset + [feature]

            # check if this subset has already been evaluated
            for node in visited:
                if (set(node) == set(temp_subset)):
                    break
            # if not, ...
            else:
                # ... mark it as visited
                visited.append( temp_subset )
                # ... compute merit
                merit = getMerit(temp_subset, label)
                # and push it to the queue
                queue.push(temp_subset, merit)
    return queue.queue[-1][0]

In [8]:
for i in range(1,14):
    print('i:',i)
    #Change the filepath to the directory containing the datasets
    df = pd.read_csv('Datasets/D'+str(i)+'.csv', header=None)
    features = list(df.columns)[:-1]
    label = list(df.columns)[-1]
    selected = cfs(features, label, df)
    print('selected:', selected)
    with open('cfs_results/D'+str(i)+'.txt', "w+") as file:
        file.truncate(0)
        file.write(str(selected))
    

i: 1
selected: [201, 224, 104, 168, 46, 202, 14, 200, 353]
i: 2
selected: [201, 224, 104, 168, 50, 202, 14, 200, 353]
i: 3
selected: [137, 224, 104, 168, 46, 202, 14, 200, 353]
i: 4
selected: [201, 224, 104, 168, 50, 202, 14, 200, 353]
i: 5
selected: [201, 224, 104, 168, 46, 202, 14, 200, 353]
i: 6
selected: [137, 224, 104, 168, 50, 202, 14, 200, 353]
i: 7
selected: [201, 31, 104, 224, 340, 202, 14, 195, 0, 353]
i: 8
selected: [137, 31, 104, 224, 340, 201, 14, 28, 202, 193, 353]
i: 9
selected: [201, 31, 104, 224, 340, 202, 14, 29, 197, 353]
i: 10
selected: [201, 31, 104, 224, 340, 137, 14, 29, 193, 353]
i: 11
selected: [201, 31, 104, 224, 340, 137, 14, 29, 193, 353]
i: 12
selected: [137, 31, 104, 224, 340, 202, 14, 29, 199, 353]
i: 13
selected: [10, 18, 92, 11, 35, 91, 125, 96, 8, 37, 22, 218]
